In [93]:
import pandas as pd
import numpy as np
import joblib

In [94]:
data=pd.read_csv('/home/thiri/Documents/rt_model/data/nov_april_prepared_data.csv')

model = joblib.load('/home/thiri/Documents/rt_model/saved_model/decision_tree_model.pkl') #Change models as needed

/home/thiri/Documents/rt_model/.venv/lib/python3.12/site-packages/sklearn/base.py:440: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.6.1 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


### **load actual May terminated/active data**

In [95]:
real_may_terminated=pd.read_csv('/home/thiri/Documents/rt_model/actual_may_terminated_active_data/id_list_terminated_in_May.csv')
real_may_active=pd.read_csv('/home/thiri/Documents/rt_model/actual_may_terminated_active_data/id_list_active_in_May.csv')

### **Filtering 'data'** 

In [96]:
real_status_data=pd.concat([real_may_terminated, real_may_active])

In [97]:
real_status_data.shape

(121169, 3)

In [98]:
real_status_data['Customer ID'].nunique()

121169

In [99]:
df=data[data['Customer ID'].isin(real_status_data['Customer ID'])]

In [100]:
df.shape

(120593, 14)

In [101]:
priority_map = {'Low': 0, 'Medium': 1, 'High': 2}
df['Last Priority'] = df['Last Priority'].map(priority_map)

df['device_growth_ratio'] = df['avg_device_count_last_2_months'] / (df['overall_avg_device_count'] + 1e-5)

df['avg_usage_per_device'] = df['Average Data Usage'] / (df['overall_avg_device_count'] + 1e-5)

/tmp/ipykernel_343712/3369433123.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Last Priority'] = df['Last Priority'].map(priority_map)
/tmp/ipykernel_343712/3369433123.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['device_growth_ratio'] = df['avg_device_count_last_2_months'] / (df['overall_avg_device_count'] + 1e-5)
/tmp/ipykernel_343712/3369433123.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

In [102]:
X_new = df.loc[:, model.feature_names_in_]
y_probs = model.predict_proba(X_new)[:, 1]

# Apply binary threshold classification (e.g., 0.4, 0.5)
threshold = 0.5
predicted_status = np.where(y_probs >= threshold, 'TERMINATED', 'ACTIVE')

df['Predicted_Status'] = predicted_status

predicted_data=df[['Customer ID', 'Predicted_Status']]

/tmp/ipykernel_343712/3489120383.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Predicted_Status'] = predicted_status


In [103]:
predicted_data

,Customer ID,Predicted_Status
0,2275947,ACTIVE
1,2150547,ACTIVE
2,2178246,ACTIVE
3,2347542,ACTIVE
4,2141477,TERMINATED
...,...,...
164338,2409654,TERMINATED
164339,2406087,TERMINATED
164351,2255050,TERMINATED
164370,2459519,TERMINATED


In [104]:
predicted_data['Predicted_Status'].value_counts()

Predicted_Status
ACTIVE        82869
TERMINATED    37724
Name: count, dtype: int64

### Checking

In [109]:
predicted_terminated = predicted_data[predicted_data['Predicted_Status']=='TERMINATED']
predicted_active = predicted_data[predicted_data['Predicted_Status']=='ACTIVE']

In [110]:
correct_terminated_count=real_may_terminated[real_may_terminated['Customer ID'].isin(predicted_terminated['Customer ID'])]

In [111]:
correct_terminated_count.shape

(1044, 3)

In [112]:
correct_active_count=real_may_active[real_may_active['Customer ID'].isin(predicted_active['Customer ID'])]

In [113]:
correct_active_count.shape

(82258, 3)